<a href="https://colab.research.google.com/github/Julialunna/Artificial-Intelligence/blob/main/FL-DP-PSO-SGD/experiments/FLPSO_SGD_Clients_unbalanced_random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FLPSO-ADAM : desbalanceamento aleatório

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset, TensorDataset
from torchvision import datasets, transforms
import torch.nn.functional as F
import copy
import random
import torchvision
import torchvision.models as models
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import numpy as np
from collections import OrderedDict

In [ ]:
class MLP(nn.Module):

    def __init__(self, device, input_size=28*28, hidden_size=256, num_classes=10):
        super(MLP, self).__init__()
        self.device = device
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)
        self.to(device)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Achatar o tensor de entrada
        y = self.fc1(x)
        y = self.relu(y)
        y = self.fc2(y)
        y = self.relu(y)
        y = self.fc3(y)

        return y

In [ ]:
# Definições dos hiperparâmetros
NUM_CLIENTES = 5
NUM_PARTICULAS = 25
NUM_RODADAS = 50
NUM_DIGITOS = 10
INERCIA, C1, C2 = 0.8, 1.5,  1.9
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'training on {DEVICE}')

# Criando o modelo global
modelo_global = MLP(DEVICE)
criterio = nn.CrossEntropyLoss()

training on cuda


In [ ]:
#Seeds para reprodutibilidade
random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

class Particula:
    def __init__(self, particle_id, modelo_cliente):
        self.particle_id = particle_id
        self.device = modelo_cliente.device
        self.pesos = copy.deepcopy(modelo_cliente.state_dict())

        # Adiciona ruído leve nos pesos para quebrar simetria inicial
        for name in self.pesos:
            self.pesos[name] += 0.01 * torch.randn_like(self.pesos[name])

        self.melhor_pesos = copy.deepcopy(self.pesos)
        self.melhor_erro = float('inf')
        self.velocidade = {name: torch.zeros_like(param) for name, param in self.pesos.items()}

    def atualizar_pso(self, global_best_pesos, INERCIA, C1, C2):
        MAX_VELOCITY = 0.1  # Limite para evitar oscilações grandes
        for name in self.pesos:
            local_rand = random.random()
            global_rand = random.random()
            self.velocidade[name] = (
                INERCIA * self.velocidade[name]
                + C1 * local_rand * (self.melhor_pesos[name] - self.pesos[name])
                + C2 * global_rand * (global_best_pesos[name] - self.pesos[name])
            )

            # Clipping da velocidade
            self.velocidade[name] = torch.clamp(self.velocidade[name], -MAX_VELOCITY, MAX_VELOCITY)

            # Atualiza os pesos com a nova velocidade
            self.pesos[name] += self.velocidade[name]

    def avaliar_perda(self, modelo_cliente, criterio, dados):
        modelo_cliente.load_state_dict(self.pesos)
        modelo_cliente.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = modelo_cliente(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)


class Cliente:
    def __init__(self, cliente_id, modelo_global, dados, test, num_particulas):
        self.cliente_id = cliente_id
        self.modelo = copy.deepcopy(modelo_global)  # Cada cliente tem seu próprio modelo
        self.dados = dados
        self.test = test
        self.num_particulas = num_particulas
        self.particulas = []
        self.melhor_particula = None
        self.inicializar_particulas(num_particulas)

    def inicializar_particulas(self, num_particulas):
        """Cria um conjunto de partículas associadas ao cliente."""
        self.particulas = [Particula(i, self.modelo) for i in range(num_particulas)]

    def treinar_com_pso(self, INERCIA, C1, C2, global_best_pesos, criterio):
        """Treina as partículas usando PSO e atualiza a melhor partícula local."""

        for particula in self.particulas:

            particula.atualizar_pso(global_best_pesos, INERCIA, C1, C2)
            erro = particula.avaliar_perda(self.modelo, criterio, self.dados)
            if erro < particula.melhor_erro:
                particula.melhor_erro = erro
                particula.melhor_pesos = copy.deepcopy(particula.pesos)

        self.selecionar_melhor_particula()

    def refinar_com_adam(self, criterio):
        """Refina os pesos da melhor partícula usando Adam."""
        self.modelo.load_state_dict(self.melhor_particula.melhor_pesos)
        otimizador = optim.Adam(self.modelo.parameters(), lr=0.007, weight_decay=1e-5)

        self.modelo.train()
        for i in range(1):
          for inputs, labels in self.dados:
              inputs, labels = inputs.to(self.modelo.device), labels.to(self.modelo.device)
              otimizador.zero_grad()
              outputs = self.modelo(inputs)
              loss = criterio(outputs, labels)
              loss.backward()
              otimizador.step()

        # Atualiza os pesos da melhor partícula com os pesos refinados pelo Adam
        self.melhor_particula.melhor_pesos = copy.deepcopy(self.modelo.state_dict())
        self.melhor_particula.melhor_erro = self.calcular_loss(self.modelo, criterio, self.dados)

    def calcular_loss(self, modelo, criterio, dados):
        self.modelo.eval()
        total_loss = 0

        with torch.no_grad():
            for inputs, labels in dados:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = modelo(inputs)
                loss = criterio(outputs, labels)
                total_loss += loss.item()

        return total_loss / len(dados)

    def selecionar_melhor_particula(self):
        """Seleciona a melhor partícula do cliente."""
        self.melhor_particula = min(self.particulas, key=lambda p: p.melhor_erro)


def treinar_federado(modelo_global, clientes, criterio, num_rodadas, INERCIA, C1, C2, testloader):
    """Treina os clientes localmente e sincroniza com o servidor central, validando a acurácia."""

    melhor_peso_global = copy.deepcopy(modelo_global.state_dict())  # Inicializa com o modelo global
    melhor_erro_global = float('inf')
    soma = 0.0;

    for rodada in range(num_rodadas):
        resultados_rodada = []

        for cliente in clientes:
            cliente.treinar_com_pso(INERCIA, C1, C2, melhor_peso_global, criterio)  # Treino com PSO
            cliente.refinar_com_adam(criterio)  # Refinamento com Adam
            erro_cliente = cliente.melhor_particula.melhor_erro  # Obtém o melhor erro do cliente
            resultados_rodada.append((cliente.cliente_id, erro_cliente))

        resultados_sorted = sorted(resultados_rodada, key=lambda x: x[1])
        top_3_results = resultados_sorted[:3]

        melhor_cliente = random.choice(top_3_results)
        melhor_cliente_id = melhor_cliente[0]
        melhor_erro_cliente = melhor_cliente[1]

        melhor_peso_global = copy.deepcopy(clientes[melhor_cliente_id].melhor_particula.melhor_pesos)
        melhor_erro_global = melhor_erro_cliente

        modelo_global.load_state_dict(melhor_peso_global)

        test_loss, test_accuracy = avaliar_modelo(modelo_global, criterio, testloader)
        soma += test_accuracy

        # if (rodada+1) % 10 == 0:
        print(f"Rodada {rodada+1}/{num_rodadas}: Cliente {melhor_cliente_id} enviou os pesos!")
        print(f"Erro Global Atualizado: {melhor_erro_global:.4f}")
        print(f"Teste -> Perda: {test_loss: .4f}, Acurácia: {test_accuracy*100: .2f}%\n")

    print(f"Acurácia média: {soma/num_rodadas: .4f}")
    print("Treinamento Federado Finalizado!")

def avaliar_modelo(modelo, criterio, testloader):
    """Avalia o modelo global no conjunto de teste."""
    modelo.eval()  # Modo de avaliação
    total_loss = 0
    correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = modelo(inputs)
            loss = criterio(outputs, labels)

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total_samples += labels.size(0)

    test_loss = total_loss / len(testloader)
    test_accuracy = (correct / total_samples)

    return test_loss, test_accuracy

mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True)
X_test = mnist_test.data.view(-1, 28*28).numpy()  # Flatten
y_test = mnist_test.targets.numpy()

mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True)
X_train = mnist_train.data.view(-1, 28*28).numpy()  # Flatten
y_train = mnist_train.targets.numpy()

# Normalização (como foi feito com o Iris)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Converter para tensores
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Criar datasets
trainset = TensorDataset(X_train, y_train)
testset = TensorDataset(X_test, y_test)

def create_unbalanced_subsets(dataset, num_clients, min_samples=300, max_samples=3500, min_per_digit=200):
    targets = dataset.tensors[1].numpy()

    # Índices das classes disponíveis
    class_indices = {digit: np.where(targets == digit)[0] for digit in range(10)}
    for digit in class_indices:
        np.random.shuffle(class_indices[digit])

    # Índices dos clientes
    client_indices = {i: [] for i in range(num_clients)}

    # Distribuição
    for digit in range(10):
        available_indices = class_indices[digit]

        # Verifica se existem amostras suficientes
        min_total_samples = num_clients * min_per_digit
        if len(available_indices) < min_total_samples:
            raise ValueError(f"Número insuficiente de amostras para o dígito {digit}")

        # Distribui inicialmente "min_per_digit" amostras por cliente
        for client_id in range(num_clients):
            start_idx = client_id * min_per_digit
            end_idx = start_idx + min_per_digit
            selected = available_indices[start_idx:end_idx]
            client_indices[client_id].extend(selected.tolist())

        available_indices = available_indices[min_total_samples:]

        # Distribui o restante das amostras aleatoriamente respeitando min_samples e max_samples
        for client_id in range(num_clients):
            if len(available_indices) == 0:
                break

            additional_samples = np.random.randint(min_samples - min_per_digit, max_samples - min_per_digit + 1)
            additional_samples = min(additional_samples, len(available_indices))

            selected = available_indices[:additional_samples]
            client_indices[client_id].extend(selected.tolist())

            available_indices = available_indices[additional_samples:]

    # Criar subsets
    subsets = [Subset(dataset, client_indices[i]) for i in range(num_clients)]

    # Mostrar distribuição final
    for client_id in range(num_clients):
        client_targets = targets[client_indices[client_id]]
        digit_counts = {digit: np.sum(client_targets == digit) for digit in range(10)}
        print(f"Cliente {client_id}: {digit_counts}")

    return subsets

train_subsets = create_unbalanced_subsets(trainset, NUM_CLIENTES)

# Criar DataLoaders para cada subset (cliente)
trainloaders = [DataLoader(train_subsets[i], batch_size=240, shuffle=True) for i in range(NUM_CLIENTES)]

testloader = DataLoader(testset, batch_size=240, shuffle=False)

# Criando os clientes
clientes = [Cliente(i, modelo_global, trainloaders[i], testloader, NUM_PARTICULAS) for i in range(NUM_CLIENTES)]

# Executando o treinamento federado
treinar_federado(modelo_global, clientes, criterio, NUM_RODADAS, INERCIA, C1, C2, testloader)

Cliente 0: {0: np.int64(949), 1: np.int64(2581), 2: np.int64(1342), 3: np.int64(1262), 4: np.int64(962), 5: np.int64(2434), 6: np.int64(1581), 7: np.int64(1449), 8: np.int64(3106), 9: np.int64(1174)}
Cliente 1: {0: np.int64(2070), 1: np.int64(895), 2: np.int64(854), 3: np.int64(1115), 4: np.int64(2541), 5: np.int64(2387), 6: np.int64(3010), 7: np.int64(2413), 8: np.int64(1714), 9: np.int64(3419)}
Cliente 2: {0: np.int64(2504), 1: np.int64(782), 2: np.int64(1241), 3: np.int64(2065), 4: np.int64(1884), 5: np.int64(200), 6: np.int64(927), 7: np.int64(2003), 8: np.int64(363), 9: np.int64(956)}
Cliente 3: {0: np.int64(200), 1: np.int64(1701), 2: np.int64(1369), 3: np.int64(1489), 4: np.int64(255), 5: np.int64(200), 6: np.int64(200), 7: np.int64(200), 8: np.int64(468), 9: np.int64(200)}
Cliente 4: {0: np.int64(200), 1: np.int64(783), 2: np.int64(1152), 3: np.int64(200), 4: np.int64(200), 5: np.int64(200), 6: np.int64(200), 7: np.int64(200), 8: np.int64(200), 9: np.int64(200)}
Rodada 1/50: Cl

In [ ]:
accuracies = [0.9375, 0.9356, 0.9315, 0.9345, 0.9259, 0.9315, 0.9340,  0.9304, 0.8951, 0.9366]
mean_acc = np.mean(accuracies)
std_acc = np.std(accuracies)

print(f"Acurácia média: {mean_acc * 100:.2f}% ± {std_acc * 100:.2f}%")

Acurácia média: 92.93% ± 1.18%
